In [1]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcd = o3d.io.read_point_cloud("final_cropped.pcd")

pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])


In [3]:
def detect_ground_plane(pcd):
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    
    return plane_model
    
    

In [4]:
def compute_plane_transformation(plane1, plane2):
    """
    Computes the transformation between two plane equations of the form ax + by + cz + d = 0.

    Args:
        plane1 (tuple): Coefficients of the first plane equation (a1, b1, c1, d1).
        plane2 (tuple): Coefficients of the second plane equation (a2, b2, c2, d2).

    Returns:
        np.ndarray: A 4x4 transformation matrix.
    """
    # Extract coefficients of the first plane equation
    a1, b1, c1, d1 = plane1

    # Extract coefficients of the second plane equation
    a2, b2, c2, d2 = plane2

    # Compute the transformation matrix coefficients
    a = a2 * a1 + b2 * b1 + c2 * c1
    b = a2 * d1 - a1 * d2
    c = b2 * d1 - b1 * d2
    d = c2 * d1 - c1 * d2

    transformation_matrix = np.array([[a, 0, 0, b],
                                      [0, a, 0, c],
                                      [0, 0, a, d],
                                      [0, 0, 0, 1]])

    return transformation_matrix

In [5]:
ground_plane = np.asarray([0,0,1,0])
pcd_ground_plane = detect_ground_plane(pcd)
trans = compute_plane_transformation( pcd_ground_plane, ground_plane)

pcd_t = cp.deepcopy(pcd).transform(trans)
pcd_t.paint_uniform_color((1,0,0))
o3d.visualization.draw_geometries([pcd, pcd_t])

In [6]:
o3d.io.write_point_cloud("final_cropped_ground_align.pcd", pcd_t)

True